## Custom Sampling / Initial Population Creation

Creating a 2d array initial population.
1. no of weeks:4
2. no of category: 1
3. no of sku:5

so it will have the shape of 20, 6


**Skipped Constraints:**
1. Max promotions duration: 4 ie. the price options can only be 1 for at most 4 times consecutively # to remove? hard to model
2. Promotion Cooldown:2 there has to be a cooldown of 2 weeks # todo to remove??

**Modelled Constraints:**
- sku level constraints
   1. Pricing options can only be all 0 or at most one 1.
   2. display and feature can only be 1 if there is 1 pricing options true
- week level constraints
   1. promo upper limit per category: 3
   2. display upper limit per category : 1
   3. feature upper limit per category: 2


In [161]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np
from pymoo.core.sampling import Sampling

from pymoo.operators.crossover.pntx import PointCrossover, SinglePointCrossover, TwoPointCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.core.mutation import Mutation
from pymoo.core.crossover import Crossover

POP_SIZE = 10
CFG = {
    'sku_num': 36,                   # no of sku
    'h': 8,                         # week horizon
    'price_opt_num': 4,             # num of pricing options dvar
    'ndf': 2,                       # num of display, feature dvar
    'lim_pro_per_cate_xu': 12,       # num of promotion items upper bound
    'lim_dis_per_cate_xu': 7,       # num of display items upper bound
    'lim_fea_per_cate_xu': 10,       # num of feature items upper bound
    'constraint_num': 2,            # simplify to 2 high level constraints
}    

# (20, )
# PRICE_LIST = np.array([200, 190, 180, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10])
def init_price_list(x, y):
    size = x * y
    values = np.arange(size * 10, 0, -10)
    return values

# TODO: integrate with demand func
PRICE_LIST = init_price_list(CFG["sku_num"], CFG["h"])

class SKUPopulationSampling(Sampling):
    """create intial population of all feasible solutions"""

    def __init__(self, cfg, pop_size):
        super().__init__()
        self.cfg = cfg
        self.pop_size = pop_size

    def _do(self, problem=None, n_samples=None, **kwargs):
        pop_size = self.pop_size
        nc = self.cfg['sku_num']
        h = self.cfg['h']
        price_ga = self.cfg['price_opt_num']
        ndf = self.cfg['ndf']
        lim_pro_per_cate_xu = self.cfg['lim_pro_per_cate_xu']
        lim_dis_per_cate_xu = self.cfg['lim_dis_per_cate_xu']
        lim_fea_per_cate_xu = self.cfg['lim_fea_per_cate_xu']
        
        # (population_size, no. of cate * weeks, no. of dvar per sku) 2, 10, 6
        # sku_pop = np.zeros((pop_size, nc*h, (price_ga + ndf)), dtype=int)
        # need to flatten for GA to work
        sku_pop = np.zeros((pop_size, nc*h * (price_ga + ndf)), dtype=int)
        

        for s in range(pop_size):
            x_sku_pop = np.zeros((nc*h, (price_ga + ndf)), dtype=int)
            
            d_nt = np.zeros(nc, dtype=int)
            f_nt = np.zeros(nc, dtype=int)
            # p_nt = np.zeros(nc, dtype=int)
            
            start_row = 0
            for pop_t in range(h):
                end_row = start_row + nc
             
                # no of promo per week at most Lim_promo_duration_upper
                sku_indices = np.random.permutation(nc)

                # Determine the number of SKUs on discount for this week
                num_discounted = min(lim_pro_per_cate_xu, nc)

                # Update pricing options for the selected SKUs
                for sku_idx in sku_indices[:num_discounted]:
                    discount_idx = np.random.choice(price_ga)
                    x_sku_pop[start_row + sku_idx, discount_idx] = 1
                
                # Update display and feature options only for SKUs on discount
                discounted_skus = sku_indices[:num_discounted]
                sample_d = np.random.choice(discounted_skus, min(lim_dis_per_cate_xu, num_discounted), replace=False)
                sample_f = np.random.choice(discounted_skus, min(lim_fea_per_cate_xu, num_discounted), replace=False)
                
                # Update the corresponding display and feature variables
                for sku_idx in discounted_skus:
                    if sku_idx in sample_d:
                        x_sku_pop[start_row + sku_idx, price_ga] = 1  # Display
                        d_nt[sku_idx] += 1
                    if sku_idx in sample_f:
                        x_sku_pop[start_row + sku_idx, price_ga + 1] = 1  # Feature
                        f_nt[sku_idx] += 1
                
                start_row = end_row
            
            # print(x_sku_pop)
            # populate solution to population
            # sku_pop[s, :, :] = x_sku_pop
            # need to flatten for GA to work
            sku_pop[s, :] = x_sku_pop.flatten()
        
        return sku_pop

class SwapMutation(Mutation):
    def __init__(self, cfg, prob=0.9):
        super().__init__()
        self.prob = prob
        self.cfg = cfg
        self.rows = self.cfg["sku_num"] * self.cfg["h"]

    def _do(self, problem, pop, **kwargs):
        # print(type(pop), pop.shape, pop)
        pop_reshaped = pop.reshape((-1,self.rows,self.cfg["price_opt_num"] + self.cfg["ndf"]))

        # print(f"{len(pop)=}, {pop_reshaped.shape=}")
        # no of candidates to mutate
        num_to_mutate = int(len(pop) * self.prob)
        # print(f"{num_to_mutate=}")

        # random select indices of candidates to mutate
        indices_to_mutate = np.random.choice(len(pop), num_to_mutate, replace=False)        

        for idx in indices_to_mutate:
            # Randomly choose two rows to swap
            row_idx1, row_idx2 = np.random.choice(self.rows, 2, replace=False)

            # Swap the rows
            pop_reshaped[idx, [row_idx1, row_idx2]] = pop_reshaped[idx, [row_idx2, row_idx1]]


        mutated_pop = pop_reshaped.reshape((-1, pop.shape[1]))
        return mutated_pop

class SwapCrossover(Crossover):
    def __init__(self, cfg, prob=0.9, n_rows_to_swap=1, n_offsprings=2):
        super().__init__(2, 2)
        self.prob = prob
        self.cfg = cfg
        self.rows = self.cfg["sku_num"] * self.cfg["h"]
        self.n_rows_to_swap = n_rows_to_swap

    def _do(self, problem, X, **kwargs):
        p1, p2 = X

        # Reshape parents into 3D arrays
        p1_reshaped = p1.reshape((-1, self.rows, self.cfg["price_opt_num"] + self.cfg["ndf"]))
        p2_reshaped = p2.reshape((-1, self.rows, self.cfg["price_opt_num"] + self.cfg["ndf"]))

        # Randomly select solutions to perform crossover
        do_crossover = np.random.random() < self.prob

        if do_crossover:
            # Randomly choose rows to swap
            row_indices = np.random.choice(self.rows, self.n_rows_to_swap, replace=False)

            # Swap the rows between parents
            for row_idx in row_indices:
                p1_reshaped[0, row_idx], p2_reshaped[0, row_idx] = p2_reshaped[0, row_idx], p1_reshaped[0, row_idx]

        Q = np.copy(X)

        # Reshape back to 2D arrays
        Q[0] = p1_reshaped.reshape(-1, p1.shape[1])
        Q[1] = p2_reshaped.reshape(-1, p2.shape[1])

        return Q    
    

class PromotionOptimizationProblem(ElementwiseProblem):
    def __init__(self):
        self.cfg = CFG
        self.sku_ndvar = (self.cfg["price_opt_num"]+self.cfg["ndf"]) # no of dvar per sku, cols
        self.rows = self.cfg["sku_num"] * self.cfg["h"] # no of rows in the 2d array of candidate soln
        self.n_var = self.rows * self.sku_ndvar
        self.sku_num = self.cfg["sku_num"]
        self.price_opt_num = self.cfg["price_opt_num"]

        super().__init__(n_var=self.n_var, n_obj=1, n_constr=self.cfg["sku_num"] * self.cfg["h"] *self.cfg["constraint_num"], 
        xl=np.zeros(self.n_var), xu=np.ones(self.n_var))

    def _evaluate(self, x, out, *args, **kwargs):
        can_sol = x.reshape((self.rows, self.sku_ndvar)) # candidate solution
        
        ### CONSTRAINTS
        cv = self._calculate_constraints(can_sol)
        out["G"] = cv

        # If there are any violations, set the objective value to a large negative value
        if np.any(cv > 0):
            out["F"] = -np.inf
            return
        
        ### PROFIT CALCULATION
        profit = self._calculate_profit(can_sol)
        out["F"] = -profit

    def _calculate_constraints(self, can_sol):
        # cv: constraints per sku
        cv1 = np.zeros(can_sol.shape[0], dtype=int)
        for i, row in enumerate(can_sol):
            # constraint 1: at most 1 price flag
            if np.sum(row[:self.price_opt_num]) not in [0, 1]:
                cv1[i] = 1
            # constraint 2: no display or feature if no promo
            if np.all(row[:self.price_opt_num] == 0) and (row[4] == 1 or row[5]==1):
                cv1[i] = 1
        
        # relax constraints work for default mutation, crossover
        # cv = cv1
        #cv2: constraints per week
        cv2 = np.zeros(can_sol.shape[0], dtype=int)
        promo_count = display_count = feature_count = 0
        for i in range(0, len(can_sol), self.sku_num):
            for row in can_sol[i:i+self.sku_num]:
                promo_count += np.sum(row[:self.price_opt_num])
                display_count += np.sum(row[self.price_opt_num:5])
                feature_count += np.sum(row[5:])
                # print(promo_count, display_count, feature_count)
            # promo count must be less than lim_pro_per_cate_xu
            if promo_count > self.cfg['lim_pro_per_cate_xu']:
                # print(promo_count)
                for j in range(i, i+self.sku_num):
                    cv2[j] = 1
            # dis count must be less than lim_dis_per_cate_xu
            elif display_count > self.cfg['lim_dis_per_cate_xu']:
                # print(display_count)
                for j in range(i, i+self.sku_num):
                    cv2[j] = 1      
            # fea count must be less than lim_fea_per_cate_xu
            elif feature_count > self.cfg['lim_fea_per_cate_xu']:
                # print(display_count)
                for j in range(i, i+self.sku_num):
                    cv2[j] = 1                    
        # Combine all constraint violations
        cv = np.concatenate((cv1, cv2))
        return cv

    def _calculate_profit(self, can_sol):
        profit = 0
        discounted_values = np.zeros_like(PRICE_LIST)

        # For loop to cater to demand function calculation
        for i in range(len(can_sol)):
            if np.all(can_sol[i, :self.price_opt_num] == 0):
                discounted_values[i] = PRICE_LIST[i]
            else:
                discount_factor = 0.8 if np.array_equal(can_sol[i, :4], [0, 0, 0, 1]) else \
                                0.6 if np.array_equal(can_sol[i, :4], [0, 0, 1, 0]) else \
                                0.4 if np.array_equal(can_sol[i, :4], [0, 1, 0, 0]) else \
                                0.2 if np.array_equal(can_sol[i, :4], [1, 0, 0, 0]) else 1
                discounted_values[i] = discount_factor * PRICE_LIST[i]

        # Calculate profit
        profit = np.sum(discounted_values)
        return profit


problem = PromotionOptimizationProblem()
algorithm = GA(pop_size=100, 
               sampling=SKUPopulationSampling(cfg=problem.cfg, pop_size=POP_SIZE),
               mutation = SwapMutation(cfg=problem.cfg, prob=0.5),
               crossover=SwapCrossover(cfg=problem.cfg, prob=0.5, n_rows_to_swap=2),
            #    crossover=PointCrossover(prob=0.8, n_points=2),
            #    mutation = PolynomialMutation(prob=0.3, repair=RoundingRepair()),
               eliminate_duplicates=True
               )

res = minimize(problem,
               algorithm,
               ('n_gen', 50),
               seed=2,
               save_history=True,
               verbose=True)



n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  2.520000E+02 |  2.520000E+02 |             - |             -
     2 |      110 |  2.520000E+02 |  2.520000E+02 |             - |             -
     3 |      210 |  2.520000E+02 |  2.520000E+02 |             - |             -
     4 |      310 |  2.520000E+02 |  2.520000E+02 |             - |             -
     5 |      410 |  2.520000E+02 |  2.520000E+02 |             - |             -
     6 |      510 |  2.520000E+02 |  2.520000E+02 |             - |             -
     7 |      610 |  2.520000E+02 |  2.520000E+02 |             - |             -
     8 |      710 |  2.520000E+02 |  2.520000E+02 |             - |             -
     9 |      810 |  2.520000E+02 |  2.520000E+02 |             - |             -
    10 |      910 |  2.520000E+02 |  2.520000E+02 |             - |             -
    11 |     1010 |  2.520000E+02 |  2.520000E+02 |             - |             -
    12 |     111

In [49]:
res.history

In [150]:
res.F

In [149]:
res.X

In [43]:
np.set_printoptions(threshold=np.inf)
res.pop.get("X")

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0,

## Draft Functions

In [24]:
import numpy as np

np.random.seed(42)

# Example arrays
# a = np.random.randint(0, 2, size=(20, 6))  # Example array a with shape (20, 6)
# b = np.array([200, 190, 180, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10])  # Example array b with shape (20,)

# a = np.random.randint(0, 2, size=(5, 6))  # Example array a with shape (20, 6)
a = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
])
b = np.array([200, 190, 180, 170, 160])  # Example array b with shape (20,)


print(a)
# Create a mask to identify rows where the first 4 values are all zero
zero_rows_mask = np.all(a[:, :4] == 0, axis=1)

# Create a mask for different discount scenarios
discount_masks = [
    np.all(a[:, :4] == np.array([0, 0, 0, 1]), axis=1),
    np.all(a[:, :4] == np.array([0, 0, 1, 0]), axis=1),
    np.all(a[:, :4] == np.array([0, 1, 0, 0]), axis=1),
    np.all(a[:, :4] == np.array([1, 0, 0, 0]), axis=1)
]

# Apply the corresponding discount to each row based on the mask
discounted_values = np.zeros_like(b)
discounted_values[zero_rows_mask] = b[zero_rows_mask]
discounted_values[discount_masks[0]] = 0.8 * b[discount_masks[0]]
discounted_values[discount_masks[1]] = 0.6 * b[discount_masks[1]]
discounted_values[discount_masks[2]] = 0.4 * b[discount_masks[2]]
discounted_values[discount_masks[3]] = 0.2 * b[discount_masks[3]]

# Calculate profit
profit = np.sum(discounted_values)

print("Profit:", profit)


[[0 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]]
Profit: 560


In [80]:
# Sample can_sol array
can_sol_sample = np.array([
    [1, 1, 1, 1, 0, 0],  # Item 1: Too many price options VIOLATED
    [0, 0, 1, 1, 1, 0],  # Item 1: Too many price options VIOLATED
    [0, 0, 0, 0, 1, 0],  # Item 1: No promotion, on display VIOLATED
    [0, 0, 0, 0, 0, 1],  # Item 1: No promotion, on feature VIOLATED
    [0, 0, 0, 0, 1, 1],  # Item 1: No promotion, on display, feature VIOLATED
    [0, 1, 0, 0, 1, 1],  # Item 2: Promotion, on display, no feature
    [1, 0, 0, 0, 0, 1],  # Item 3: Promotion, not on display
    [1, 0, 0, 0, 1, 0],  # Item 3: Promotion, not on feature
    [0, 0, 0, 1, 1, 0],  # Item 4: promotion, on display
    [0, 1, 0, 0, 1, 1]   # Item 5: Promotion, on display, on feature
])

# Calculate constraint violation cv
cv = np.zeros(can_sol_sample.shape[0], dtype=int)
for i, row in enumerate(can_sol_sample):
    print(row[:4], row[4])
    # constraint 1: at most 1 price flag
    if np.sum(row[:4]) not in [0, 1]:
        cv[i] = 1
    # constraint 2: no display or feature if no promo
    if np.all(row[:4] == 0) and (row[4] == 1 or row[5]==1):
        cv[i] = 1
    

print("Constraint violation cv:", cv)




can_sol_sample2 = np.array([
    [0, 0, 1, 0, 1, 0], 
    [0, 1, 0, 0, 0, 1], 
    [1, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 1, 0], 
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 0, 1, 0], 
    [0, 1, 0, 0, 0, 1], 
    [1, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 1, 0], 
    [0, 1, 0, 0, 1, 1]     
])

can_sol_sample3 = np.array([
    [0, 0, 0, 0, 1, 0], 
    [0, 1, 0, 0, 0, 1], 
    [1, 0, 0, 0, 0, 1], 
    [0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0], 
    [0, 1, 0, 0, 0, 1], 
    [1, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 1, 1]     
])

can_sol_sample2 =can_sol_sample3

# Calculate constraint violation cv2
cv2 = np.zeros(can_sol_sample2.shape[0], dtype=int)
promo_count = display_count = feature_count = 0
for i in range(0, len(can_sol_sample2), 5):
    for row in can_sol_sample2[i:i+5]:
        promo_count += np.sum(row[:4])
        display_count += np.sum(row[4:5])
        feature_count += np.sum(row[5:])
        print(promo_count, display_count, feature_count)
    if promo_count > 3:
        print(promo_count)
        for j in range(i, i+5):
            cv2[j] = 1
    elif display_count > 1:
        print(display_count)
        for j in range(i, i+5):
            cv2[j] = 1      
    elif feature_count > 2:
        print(display_count)
        for j in range(i, i+5):
            cv2[j] = 1                  

    promo_count = display_count = feature_count = 0

print("Constraint violation cv2:", cv2)


[1 1 1 1] 0
[0 0 1 1] 1
[0 0 0 0] 1
[0 0 0 0] 0
[0 0 0 0] 1
[0 1 0 0] 1
[1 0 0 0] 0
[1 0 0 0] 1
[0 0 0 1] 1
[0 1 0 0] 1
Constraint violation cv: [1 1 1 1 1 0 0 0 0 0]
0 1 0
1 1 1
2 1 2
2 2 2
2 3 3
3
0 1 0
1 1 1
2 1 1
2 2 1
2 3 2
3
Constraint violation cv2: [1 1 1 1 1 1 1 1 1 1]


In [98]:
import numpy as np
np.random.seed(42)
# Assume you have a 5x30 array of candidate solutions with values 0 or 1
candidates = np.random.randint(2, size=(5, 30))
print(candidates.shape[1])

# Reshape each candidate solution into a 5x5x6 array
candidates_reshaped = candidates.reshape((5, 5, 6))
print(f"{candidates_reshaped=}")

# Probability of mutation (e.g., 50%)
mutation_prob = 0.5

# Determine the number of candidates to mutate based on the probability
num_to_mutate = int(len(candidates) * mutation_prob)
print(f"{num_to_mutate=}")

# Randomly select indices of candidates to mutate
indices_to_mutate = np.random.choice(len(candidates), num_to_mutate, replace=False)
print(f"{indices_to_mutate=}")

# Loop through selected candidates and perform mutation
for idx in indices_to_mutate:
    # Randomly choose two rows to swap
    row_idx1, row_idx2 = np.random.choice(5, 2, replace=False)
    print(f"{row_idx1=}, {row_idx2=}")
    
    # Swap the rows
    candidates_reshaped[idx, [row_idx1, row_idx2]] = candidates_reshaped[idx, [row_idx2, row_idx1]]
print(f"{candidates_reshaped=}")
# Reshape the mutated candidates back to the original shape
mutated_candidates = candidates_reshaped.reshape((5, 30))



30
candidates_reshaped=array([[[0, 1, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 1, 1],
        [1, 0, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1]],

       [[0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 0, 1, 0, 1],
        [0, 1, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 1],
        [1, 0, 1, 1, 1, 1],
        [0, 1, 0, 1, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1]],

       [[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 1],
        [0, 1, 0, 1, 1, 0]],

       [[1, 0, 1, 0, 0, 1],
        [1, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 1],
        [1, 0, 0, 0, 0, 1]]])
num_to_mutate=2
indices_to_mutate=array([1, 3])
row_idx1=0, row_idx2=3
row_idx1=1, row_idx2=4
candidates_reshaped=array([[[0, 1, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 1, 1],
        [1, 0, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1]],

       [[1,

In [91]:
mutated_candidates


array([[3.95573491e-01, 1.64304472e-01, 5.38218995e-01, 1.78354285e-01,
        4.90443469e-02, 6.14339221e-01, 3.81005485e-01, 6.83474893e-01,
        5.22503225e-01, 6.27631073e-01, 3.18492512e-01, 5.82227684e-01,
        4.94572992e-01, 5.94317008e-01, 7.68368204e-03, 9.64897912e-01,
        2.35107544e-01, 7.66856222e-02, 2.94822451e-01, 6.92934647e-01,
        9.32733472e-01, 8.44150941e-01, 3.26693625e-01, 8.32170922e-01,
        5.43958369e-02, 9.28304637e-02, 1.86347530e-01, 7.87766346e-01,
        7.24243816e-01, 1.31137839e-01, 7.08128640e-01, 7.61778181e-01,
        1.90860452e-01, 6.70697174e-01, 4.05600567e-01, 7.34079817e-02,
        5.17454510e-01, 4.92567201e-01, 8.81908698e-01, 5.60191724e-01,
        8.40248732e-01, 4.53256600e-01, 7.15539575e-01, 9.69863999e-01,
        2.11155417e-02, 4.51306486e-01, 4.29854276e-01, 7.80069991e-01,
        1.85384988e-02, 5.79852857e-01, 2.78824317e-01, 4.76578522e-01,
        7.17254424e-01, 6.29896041e-01, 2.46196409e-01, 2.054060

In [136]:
import numpy as np

# Example arrays
p1_reshaped = np.array([[[1, 1, 1], [2, 2, 2], [3, 3, 3]]])
p2_reshaped = np.array([[[4, 4, 4], [5, 5, 5], [6, 6, 6]]])

print("\nBefore swap:")
print("Parent 1:")
print(p1_reshaped)
print("\nParent 2:")
print(p2_reshaped)


# Randomly choose two rows to swap
row_idx1, row_idx2 = np.random.choice(3, 2, replace=False)
print(f"Original rows: {row_idx1}, {row_idx2}")

# Swap the rows between parents
p1_reshaped[0, [row_idx1, row_idx2]] = p2_reshaped[0, [row_idx2, row_idx1]]
p2_reshaped[0, [row_idx1, row_idx2]] = p1_reshaped[0, [row_idx2, row_idx1]]

print("\nAfter swap:")
print("Parent 1:")
print(p1_reshaped)
print("\nParent 2:")
print(p2_reshaped)



Before swap:
Parent 1:
[[[1 1 1]
  [2 2 2]
  [3 3 3]]]

Parent 2:
[[[4 4 4]
  [5 5 5]
  [6 6 6]]]
Original rows: 0, 1

After swap:
Parent 1:
[[[5 5 5]
  [4 4 4]
  [3 3 3]]]

Parent 2:
[[[4 4 4]
  [5 5 5]
  [6 6 6]]]
